In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as k
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras.engine.topology import Layer
from keras import initializers
from keras.models import Sequential
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#global variables
global t, x, y, z
t = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = tf.placeholder(tf.float32)

In [ ]:
class second_Dense(Layer):
  def __init__(self, n_units, **kwargs):
    self.n_units = n_units
    super().__init__(**kwargs)
  def build(self, inshape):
    self.w = self.add_weight("w",(inshape[1], self.n_units),initializer = initializers.get('glorot_uniform'))
    self.g = self.add_weight("g",(inshape[1], self.n_units),initializer = initializers.get('glorot_uniform'))
    self.b = self.add_weight("b",(self.n_units,),initializer = initializers.get('zeros'))
    super().build(inshape)
  def call(self, x):
    return tf.matmul(tf.matmul(x,self.w),self.g)+self.b
  def comput_output_shape(self, inshape):
    return (inshape[0],self.n_units)
    
def sin_activation(x):
  return k.sin(x)
def Governing_Equation(p,u,v,w,Re):
  #get the x,y,z,t at the global variables.
    u_t = fwd_gradients(u,t)
    v_t = fwd_gradients(v,t)
    w_t = fwd_gradients(w,t)
    
    u_x = fwd_gradients(u,x)
    v_x = fwd_gradients(v,x)
    w_x = fwd_gradients(w,x)
    
    u_xx = fwd_gradients(u_x,x)
    v_xx = fwd_gradients(v_x,x)
    w_xx = fwd_gradients(w_x,x)
    
    u_y = fwd_gradients(u,y)
    v_y = fwd_gradients(v,y)
    w_y = fwd_gradients(w,y)
    
    u_yy = fwd_gradients(u_y,y)
    v_yy = fwd_gradients(v_y,y)
    w_yy = fwd_gradients(w_y,y)
    
    u_z = fwd_gradients(u,z)
    v_z = fwd_gradients(v,z)
    w_z = fwd_gradients(w,z)
    
    u_zz = fwd_gradients(u_z,z)
    v_zz = fwd_gradients(v_z,z)
    w_zz = fwd_gradients(w_z,z)
    
    p_x = fwd_gradients(p,x)
    p_y = fwd_gradients(p,y)
    p_z = fwd_gradients(p,z)
    
    e1 = u_t+(u*u_x + v*u_y + w*u_z) - (1.0/Re)*(u_xx + u_yy + u_zz)#Navier-stokes x
    e2 = v_t+(u*v_x + v*v_y + w*v_z) - (1.0/Re)*(v_xx + v_yy + v_zz)#Navier-stokes y
    e3 = w_t+(u*w_x + v*w_y + w*w_z) - (1.0/Re)*(w_xx + w_yy + w_zz)#Navier-stokes z
    e4 = u_x+v_y+w_z#Continuous Equation
    
    return [e1, e2, e3, e4]

def PINNs_loss(y_true,y_pred):
    #At y , [e1, e2, e3, e4, u, v] are contained as a [6,] form
    residual = Governing_Equation(y_pred[0,:],y_pred[1,:],y_pred[2,:],y_pred[3,:],y_pred[4,:])
    residual.append([y_pred[1,:],y_pred[2,:]])
    loss = k.mean(k.square(y_true-residual))
    
    return loss
    
def fwd_gradients(y, x):
    y_x = tf.gradients(y,x)
    
    return y_x
def Neural_Network():
  get_custom_objects().update({'sin_activation': Activation(sin_activation)})
  model = Sequential()
  #inputs = keras.Input(shape = (4,))#x,y,z,t
  #dense = second_Dense(50, activation = sin_activation)
  model.add(second_Dense(50, input_shape = (4,)))
  model.add(sin_activation)
  for i in range(8):
      model.add(second_Dense(50))
      model.add(sin_activation)
  model.add(second_Dense(5)) #p,u,v,w,Re

  return model

In [ ]:
model = Neural_Network()
model.compile(optimizer = 'adam', loss = PINNs_loss)
model.fit(data, labels, epochs= 100, batch_size = 32)

ValueError: ignored

In [ ]:
type(t)

tensorflow.python.framework.ops.Tensor

In [ ]:
model.summary()

Model: "Fully connected layer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_42 (Dense)             (None, 50)                250       
_________________________________________________________________
dense_43 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_44 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_45 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_46 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_47 (Dense)             (None, 50)      